In [1]:
# !pip install ipywidgets jupyterlab_widgets
# !pip install --upgrade ipywidgets jupyterlab_widgets

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, clear_output
import ipywidgets as widgets

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_bloch_vector
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import state_fidelity

<center><h1>Quantum Gates Puzzle</h1></center>

![game_image](https://miro.medium.com/v2/resize:fit:1400/1*RjuzQwddn0K_X9_z64cJBA.png)

### Objective  
Use a limited number of quantum gates to transform a starting quantum state into a goal state. Success is determined by whether the final quantum state has a high probability (≥ 90%) of measuring the goal state.

### Game Rules  
You are given a **level** with:
- A number of **qubits**
- A set of **starting gates** (applied before you begin)
- A **goal** state (e.g., |1⟩, |11⟩)
- A **move limit** (number of gates you can apply).

Your task:
- Apply a combination of allowed gates within the move limit to reach the goal state

You win if:  
- The **probability of the goal state** in the final state vector is **≥ 0.9** (90%)

### Controls & UI  
| UI Element         | Description                                                                                               |
| ------------------ | --------------------------------------------------------------------------------------------------------- |
| **Gate Dropdown**  | Select a quantum gate: `X`, `H`, `Z`, `CNOT`                                                              |
| **Target Input**   | Specify the qubit(s) to apply the gate to <br>Single target: `0` <br>For `CNOT`: `0,1` (control, target) |
| **Apply Gate**     | Apply the selected gate to the target(s)                                                                  |
| **Check Solution** | Evaluate your current quantum state                                                                       |
| **Reset Level**    | Restart from initial configuration                                                                        |

### Available Gates  
| Gate   | Description    | Effect                                       |
| ------ | -------------- | -------------------------------------------- |
| `X`    | Pauli-X        | Flips $\lvert 0\rangle$ <-> $\lvert 1\rangle$|
| `H`    | Hadamard       | Creates superposition         |       |      |
| `Z`    | Pauli-Z        | Phase flip                    |       |      |
| `CNOT` | Controlled-NOT | Flips target if control is $\lvert 1\rangle$ |

### How to Play (Step-by-Step)
1. **Read the level info:** number of qubits, starting state, goal state, and move limit.
2. **Use dropdown** to select a gate
3. **Enter qubit targets:**
    - Single-qubit gates: 0, 1, etc.
    - CNOT: control,target (e.g., 0,1)
4. **Click "Apply Gate"** - The circuit updates and one move is used.
5. **Repeat** until you:
    - Reach the goal state, or
    - Run out of moves
6. **Click "Check Solution"** to verify if you succeeded
7. **Use "Reset Level"** to try again if needed

**Example Levels**

**Level 1:** Flip Qubit
- Qubits 1
- Start: $\lvert 0\rangle$
- Goal: $\lvert 1\rangle$
- Limit: 1 gate

Solution:
- Gate: `X`
- Target: `0`
- Explanation: Pauli-X flips $\lvert 0\rangle$ -> $\lvert 1\rangle$

**Level 2:** Superposition Challenge (for future levels)
- Qubits: 1
- Start: $\lvert 0\rangle$
- Goal: 50/50 between $\lvert 0\rangle$ and $\lvert 1\rangle$
- Limit: 1 gate

Solution:
- Gate: `H`
- Target: `0`
- Explanation: Hadamard creates $(\lvert 0\rangle + \lvert 1\rangle) / \sqrt{2}$

**Tips**
- Use `H` to enter superposition, and `Z` to flip phase (for advanced logic)
- You can mix gates (e.g., `H` + `X`) for more complex transformations
- `CNOT` only works with **2 qubits** - make sure both are defined and targeted correctly
- You only get **limited moves**, so plan carefully!

*Have fun*☺️

⚠️ *Note: The implementation of the code below requires IPyWidgets. If it's missing, it can be installed from the import block at the top of the page.*

### Game Implementation

In [5]:
backend = Aer.get_backend('statevector_simulator')

# Define Game Level
class QuantumPuzzleLevel:
    def __init__(self, num_qubits, start_gates=[], goal_state=None, gate_limit=3, goal_statevector=None):
        self.num_qubits = num_qubits
        self.start_gates = start_gates  # List of (gate, target) tuples
        self.goal_state = goal_state if goal_state is not None else ['1'] * num_qubits
        self.goal_statevector = goal_statevector
        self.gate_limit = gate_limit
        self.gates_applied = []

    def create_circuit(self):
        qc = QuantumCircuit(self.num_qubits)
        for gate, target in self.start_gates:
            self.apply_gate(qc, gate, target)
        return qc

    def apply_gate(self, qc, gate, target):
        if gate == 'X':
            qc.x(target)
        elif gate == 'H':
            qc.h(target)
        elif gate == 'Z':
            qc.z(target)
        elif gate == 'CNOT':
            qc.cx(*target)
        # Add more gates here if desired

    def simulate(self, qc):
        job = execute(qc, backend)
        result = job.result()
        return result.get_statevector(qc)

# UI and Game Engine
class QuantumPuzzleGame:
    def __init__(self, level: QuantumPuzzleLevel):
        self.level = level
        self.qc = level.create_circuit()
        self.remaining_moves = level.gate_limit
        self.output = widgets.Output()

        self.gate_dropdown = widgets.Dropdown(
            options=['X', 'H', 'Z', 'CNOT'],
            description='Gate:'
        )
        self.target_input = widgets.Text(
            placeholder='e.g. 0 or 0,1 for CNOT',
            description='Target:'
        )
        self.apply_button = widgets.Button(description='Apply Gate')
        self.check_button = widgets.Button(description='Check Solution')
        self.reset_button = widgets.Button(description='Reset Level')

        self.apply_button.on_click(self.apply_gate)
        self.check_button.on_click(self.check_solution)
        self.reset_button.on_click(self.reset_game)

        self.render_ui()

    def render_ui(self):
        display(widgets.HBox([self.gate_dropdown, self.target_input, self.apply_button]))
        display(widgets.HBox([self.check_button, self.reset_button]))
        display(self.output)

    def apply_gate(self, b):
        with self.output:
            clear_output()
            if self.remaining_moves <= 0:
                print("No moves left!")
                return
            gate = self.gate_dropdown.value
            try:
                targets = [int(x.strip()) for x in self.target_input.value.split(',')]
                if gate == 'CNOT' and len(targets) != 2:
                    print("CNOT needs two targets!")
                    return
                elif gate != 'CNOT' and len(targets) != 1:
                    print(f"{gate} needs one target!")
                    return

                self.level.apply_gate(self.qc, gate, targets if gate == 'CNOT' else targets[0])
                self.level.gates_applied.append((gate, targets))
                self.remaining_moves -= 1
                print(f"Applied {gate} on {targets}. Moves left: {self.remaining_moves}")
                display(self.qc.draw('mpl', style='clifford'))
            except Exception as e:
                print(f"Error: {e}")

    def check_solution(self, b):
        with self.output:
            clear_output()
            statevector = self.level.simulate(self.qc)
            probs = np.round(np.abs(statevector) ** 2, 3)
            print("Final State Probabilities:", probs)
            
            # If goal_statevector is set, use fidelity to compare
            if hasattr(self.level, 'goal_statevector') and self.level.goal_statevector is not None:
                fidelity = state_fidelity(statevector, self.level.goal_statevector)
                print(f"Fidelity with goal state: {fidelity:.3f}")
                if fidelity > 0.9:
                    print("🟢Success! You reached the goal state.")
                else:
                    print("🔴Not yet! Try again.")
            else:
                # Otherwise fallback to old method (goal_state as bitstring)
                goal_index = int(''.join(self.level.goal_state), 2)
                if probs[goal_index] > 0.9:
                    print("🟢Success! You reached the goal state.")
                else:
                    print("🔴Not yet! Try again.")

    def reset_game(self, b):
        with self.output:
            clear_output()
            print("Resetting game...")
            self.qc = self.level.create_circuit()
            self.remaining_moves = self.level.gate_limit
            self.level.gates_applied.clear()
            display(self.qc.draw('mpl', style='clifford'))

# Goal state: superposition 50/50 between |0⟩ and |1⟩
superposition_state = Statevector([1/np.sqrt(2), 1/np.sqrt(2)])

# Goal: (|00⟩ + |11⟩) / √2
entangled_goal = Statevector([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])

### Define and Launch Levels

In [7]:
# Level 1: Target |1⟩ from |0⟩
level1 = QuantumPuzzleLevel(
    num_qubits=1,
    start_gates=[],
    goal_state=['1'],
    gate_limit=1
)

# Level 2: Target equal superposition from |0⟩
level2 = QuantumPuzzleLevel(
    num_qubits=1,
    start_gates=[],
    goal_statevector=superposition_state,
    gate_limit=1
)

# Level 3: Create entangled state |11⟩ using H and CNOT
level3 = QuantumPuzzleLevel(
    num_qubits=2,
    start_gates=[],
    goal_statevector=entangled_goal,
    gate_limit=2
)

# Launch game with а level of your choice
game = QuantumPuzzleGame(level1)

Output()